In [ ]:
import pandas as pd
import os

import config

from utils.evaluation_utils import print_ensemble_results, get_best_epoch
from utils.load_utils import load_image
from utils.label_eval_utils import *


In [ ]:
STAGE = 'ss'
#MOD_PREFIX = "mod_2"
MOD_PREFIX = "mod_st"
NEPOCH = 400
MARGIN = 0.8

#DATAPATH = config.CHENETAL_DATAPATH
DATAPATH = config.SCHULTHESS_DATAPATH

## Visualize Results AS

### KL Score vs. Anom Count

In [ ]:
SAVE_PATH = config.OUTPUT_PATH
pseudolabels_path= os.path.join(SAVE_PATH, 'pseudolabels')

try:
    pseudolabel_names = [f"{STAGE}_training_{MOD_PREFIX}_epoch_margin_{MARGIN}.csv", f"{STAGE}_training_{MOD_PREFIX}_on_test_set_margin_{MARGIN}.csv"]
    df = pd.read_csv(os.path.join(pseudolabels_path, pseudolabel_names[0]), index_col=False)
    dftest = pd.read_csv(os.path.join(pseudolabels_path, pseudolabel_names[1]), index_col=False)
except:
    pseudolabel_names = [f"{STAGE}_training_{MOD_PREFIX}_epoch_{NEPOCH}_margin_{MARGIN}.csv", f"{STAGE}_training_{MOD_PREFIX}_on_test_set_epoch_{NEPOCH}_margin_{MARGIN}.csv"]
    df = pd.read_csv(os.path.join(pseudolabels_path, pseudolabel_names[0]), index_col=False)
    dftest = pd.read_csv(os.path.join(pseudolabels_path, pseudolabel_names[1]), index_col=False)

In [ ]:
df.head()

In [ ]:
graph_save_path = os.path.join(SAVE_PATH, 'graphs', STAGE)
os.makedirs(graph_save_path, exist_ok=True)

In [ ]:
plot_comparison_boxplot(df, dftest, "anoms_count", title="Anomality Count Distribution (Log)", log=True)

In [ ]:
x = 'label'
y = 'av'
x_title = 'KL Score'
y_title = 'Average AS'

filename = f"boxplot_{pseudolabel_names[0].split('.')[0]}_KLscore_AverageAS.png"
boxplot(df, x=x, y=y, x_title=x_title,y_title=y_title, title = "Boxplot KL Score vs Average AS Train Data", save_path=os.path.join(graph_save_path, filename))

In [ ]:
filename = f"boxplot_{pseudolabel_names[1].split('.')[0]}_KLscore_AverageAS.png"
boxplot(dftest, x=x, y=y, x_title=x_title,y_title=y_title, title = "Boxplot KL Score vs Average AS Test Data", save_path=os.path.join(graph_save_path, filename))

In [ ]:
#TODO: Variability of AS over boxplots?
# l = ['col_1001', 'col_71530', 'col_138647',
#        'col_875688', 'col_985772', 'col_44', 'col_34', 'col_193', 'col_244959',
#        'col_8765']

# x = 'label'
# x_title = 'KL Score'

# #if I have a boxplot for every "seed" is there a better way of visualizing the variability that we see for the same samples across seeds?
# # specifically also for the cases where we have inconsistent anomalie "labels"

# for i in l:
#     boxplot(df, x = x)

In [ ]:
x = 'sim'
y = 'av'
x_title = 'CLIP Sim Score'
y_title = 'Average AS'

In [ ]:
d = [df, dftest]
n = ['Train', 'Test']

dataset = 'OAI' #ST

for i in range(len(d)):
    df1 = d[i]
    scatter_plot(df1[df1['anoms_count']>0], x=x, y=y, x_title=x_title,y_title=y_title, label='label', title=f"{dataset} - AS Score (>0) vs CLIP Sim Score (m=0.8) - {n[i]}")
    scatter_plot(df1[(df1['anoms_count']<=5) & (df1['anoms_count']>0)], x=x, y=y, x_title=x_title,y_title=y_title, label='label', title=f"{dataset} - AS Score (<=5, >0) vs CLIP Sim Score (m=0.8) - {n[i]}")
    scatter_plot(df1, x=x, y=y, x_title=x_title,y_title=y_title, label='label', title=f"{dataset} - AS Score vs CLIP Sim Score (m=0.8) - {n[i]}")

#### Pairplots

In [ ]:
values = ['label', 'anoms_count', 'sim', 'av']

sns.pairplot(df[values], hue='label', palette='viridis')
plt.show()

In [ ]:
sns.pairplot(dftest[values], hue='label', palette='viridis')
plt.show()

In [ ]:
# g = sns.PairGrid(df[values], hue='label')
# g.map_lower(sns.boxplot) 
# g.map_diag(sns.histplot)  # Apply histogram for the diagonal
# g.add_legend()
# plt.show()

# Example X-Rays

## High CLIP Similarity Score for Different KL scores

In [ ]:
id_l = []
for i in range(len(pseudolabel_names)):
    df1 = d[i]
    ids = df1[df1['sim>95th']>0]
    id_l.append(ids)

    file_name = pseudolabel_names[i].rsplit('.', maxsplit=1)[0] + '_' + 'simscore_over_95th.csv'
    save_path_ids = os.path.join(SAVE_PATH, 'eval', STAGE)
    os.makedirs(save_path_ids, exist_ok=True)

    ids.to_csv(os.path.join(save_path_ids, file_name), index=False)

In [ ]:
img_save_path_train = os.path.join(SAVE_PATH, 'images', STAGE, 'train')
img_save_path_test = os.path.join(SAVE_PATH, 'images', STAGE, 'test')


for i in range(len(pseudolabel_names)):
    pseudolabel_name = pseudolabel_names[i]
    ids = id_l[i]
    for i in range(0, 5):
        l_img = []
        temp_img_savepath_train = os.path.join(img_save_path_train, str(i))
        temp_img_savepath_test = os.path.join(img_save_path_test, str(i))
        os.makedirs(temp_img_savepath_train, exist_ok=True)
        os.makedirs(temp_img_savepath_test, exist_ok=True)
        for j in range(len(ids['id'])):
            #print(int(ids['label'].iloc[j]))
            if int(ids['label'].iloc[j]) == int(i):
                l_img.append(ids['id'].iloc[j])
            
        for filename in l_img:
            if "on_test_set" in pseudolabel_name:
                img = load_image(os.path.join(DATAPATH, 'test', filename))
                save_name = os.path.join(temp_img_savepath_test, f'{pseudolabel_name}_over95thpercentile{filename.split('/')[1]}')
            else:
                img = load_image(os.path.join(DATAPATH, 'train', filename))
                save_name = os.path.join(temp_img_savepath_train, f'{pseudolabel_name}_over95thpercentile{filename.split('/')[1]}')
            img.save(save_name)


SAME FOR mod_st???